In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from lib.BackTest import BackTest
from lib.models import BaseModel
from prebuilt.portfolioAllocations import PercentagePortfolioAllocation
from prebuilt.riskManagement import TakeProfitStopLoss
from prebuilt.riskManagement import TraillingStopLoss
from custom.equitiesSelection import SpacsSelectionModel
from prebuilt.strategies import Double7Signal
from custom.strategies import SpacSignals

In [2]:
# Template

# class TemplateModel(BaseModel):
#     def __init__():
#         super.__init__()
#         self.start_date = '2023-01-01'
#         self.end_date = '2024-01-01'
#         self.model_name = 'Basic Hold Model'
#         self.universe_management = BaseUniverseSetting()
#         self.equity_selection_management = BaseEquitySelectionModel()
#         self.equity_filter = BaseEquityFilter()
#         self.api_model_management = BaseAPIModel()
#         self.portfolio_management = BasePortfolioAllocation(50000)
#         self.hedging_system = None
#         self.alpha_model = BaseHold()
#         self.risk_control = BaseRiskManagement()
#         self.charge_system = BaseChargeModel()

# BackTest(TemplateModel).run()

In [3]:
class SpacInsiderLogisticRegressionModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'SPAC Insider Logistic Regression Short Model'
        self.equity_selection_management = SpacsSelectionModel(40, 5, 'Short', self.start_date, self.end_date)
        self.portfolio_management = PercentagePortfolioAllocation(1000000, 10)
        self.alpha_model = SpacSignals()
        # self.risk_control = TakeProfitStopLoss(100, 20)

Test = BackTest(SpacInsiderLogisticRegressionModel)
Test.run()

[*********************100%***********************]  70 of 70 completed


0.00858594408261078
0.005831886794142542


In [4]:
# import pandas as pd
# from json import loads, dumps
# pd.DataFrame(dict(Test.data), index=[0]).to_csv('test.csv')
# # jsonObj = loads(pd.DataFrame(, columns=[x for (x, y) in Test.data]).to_json(orient='records'))

# # with open('text.json', 'w', encoding='utf-8') as jsonf1:
# #   jsonf1.write(dumps(jsonObj, indent=4))

In [5]:
class SpacInsiderLogisticRegressionModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'SPAC Insider Logistic Regression Long Model'
        self.equity_selection_management = SpacsSelectionModel(40, 5, 'Long', self.start_date, self.end_date)
        self.portfolio_management = PercentagePortfolioAllocation(1000000, 10)
        self.alpha_model = SpacSignals()
        # self.risk_control = TakeProfitStopLoss(100, 20)

Test = BackTest(SpacInsiderLogisticRegressionModel)
Test.run()

[*********************100%***********************]  43 of 43 completed


0.016482919741814935
0.01133447660706491


In [6]:
# class Double7SignalModel(BaseModel):
#     def __init__(self):
#         super().__init__()
#         self.start_date = "2024-09-21"
#         self.end_date = "2024-10-21"
#         self.universe_management.setIntervals(['5m'])
#         self.model_name = 'ETH-USD Double 7 Signal Model'
#         self.equities = ['ETH-USD']
#         self.alpha_model = Double7Signal()

# BackTest(Double7SignalModel).run()

In [11]:
import os
import pandas as pd
from json import loads, dumps

DEFAULT_PATH = "models"
WEBPUBLICPATH = f'web/src/data/'

temporary_stonks = {}
stonks_json = {'key': 'Equities', 'value': []}
portfolio_json = {'key': 'Portfolio', 'value': []}
trading_records_json = {'key': 'Trading Records', 'value': []}
tables_json = {'key': 'Tables', 'value': []}

models = os.listdir(DEFAULT_PATH)
for model in models:
  data_path = os.path.join(DEFAULT_PATH, model, "data")
  stonks = os.listdir(data_path)

  tmp = []
  for stonk in stonks:
    stonk_json_obj = {}
    info = stonk.split("-")

    interval = info[-1].split(".")[0]
    stonk_path = os.path.join(data_path, stonk)
    df = pd.read_csv(stonk_path)
    df_obj = loads(df.to_json(orient='records'))

    stonk_json_obj['name'] = info[0]
    stonk_json_obj['value'] = df_obj
    tmp.append(stonk_json_obj)
  stonks_json['value'].append({'name': model, 'value': tmp})

  portfolio_path = os.path.join(DEFAULT_PATH, model, "summary", "portfolio.csv")
  portfolio_json_obj = {}
  df = pd.read_csv(portfolio_path)
  df_obj = loads(df.to_json(orient='records'))

  portfolio_json_obj["name"] = model
  portfolio_json_obj["value"] = df_obj
  portfolio_json["value"].append(portfolio_json_obj)

  trading_records_path = os.path.join(DEFAULT_PATH, model, "summary", "trading-records.csv")
  trading_records_json_obj = {}
  df = pd.read_csv(trading_records_path)
  df_obj = loads(df.to_json(orient='records'))

  trading_records_json_obj["name"] = model
  trading_records_json_obj["value"] = df_obj
  trading_records_json["value"].append(trading_records_json_obj)

  table_path = os.path.join(DEFAULT_PATH, model, "summary", "table.csv")
  table_json_obj = {}
  df = pd.read_csv(table_path)
  df_obj = loads(df.to_json(orient='records'))

  table_json_obj["name"] = model
  table_json_obj["value"] = df_obj
  tables_json["value"].append(table_json_obj)


for interval in temporary_stonks.keys():
  for stonk in temporary_stonks[interval].keys():
    stonks_json['value'].append({'interval': interval, 'equities': {'name': stonk, 'value': temporary_stonks[interval][stonk]}})

klineFilePath = os.path.join(WEBPUBLICPATH, 'kline.json')
with open(klineFilePath, 'w', encoding='utf-8') as jsonf1:
  jsonf1.write(dumps(stonks_json, indent=4))

recordsFilePath = os.path.join(WEBPUBLICPATH, 'portfolio.json')
with open(recordsFilePath, 'w', encoding='utf-8') as jsonf2:
  jsonf2.write(dumps(portfolio_json, indent=4))

recordsFilePath = os.path.join(WEBPUBLICPATH, 'trading-records.json')
with open(recordsFilePath, 'w', encoding='utf-8') as jsonf3:
  jsonf3.write(dumps(trading_records_json, indent=4))

tableFilePath = os.path.join(WEBPUBLICPATH, 'tables.json')
with open(tableFilePath, 'w', encoding='utf-8') as jsonf3:
  jsonf3.write(dumps(tables_json, indent=4))